In [1]:
!pip install pandas duckdb matplotlib

import pandas as pd
import duckdb
import os
import matplotlib.pyplot as plt

In [5]:
file_path = '../intel_cpus.csv'
if os.path.exists(file_path):
    df = pd.read_csv(file_path, sep=";")
else:
    raise FileNotFoundError(f"The file {file_path} does not exist.")

ParserError: Error tokenizing data. C error: Expected 8 fields in line 65, saw 9


In [3]:
con = duckdb.connect("../spec.db") # assumes DuckDB DB "spec.db" in root dir
con.sql("DROP TABLE IF EXISTS intel_cpus")
con.sql("CREATE TABLE intel_cpus AS SELECT * FROM df")
con.close()